## Goller

In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



df_goals = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\goals.csv")
df_xg = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\expected_goals_stats.csv")
df_g90 = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\goals_per_90_stats.csv")
df_bigch = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\big_chances_created_stats.csv")
df_attempts = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\total_scoring_attempts_stats.csv")
df_assists = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\assists.csv")


scaler = MinMaxScaler()
df_goals["Goals_Rating"] = scaler.fit_transform(df_goals[["Goals"]]) * 100
df_xg["xG_Rating"] = scaler.fit_transform(df_xg[["Expected_Goals"]]) * 100
df_g90["G90_Rating"] = scaler.fit_transform(df_g90[["Goals_Per_90"]]) * 100
df_bigch["BigChance_Rating"] = scaler.fit_transform(df_bigch[["Big_Chances_Created"]]) * 100
df_attempts["Attempt_Rating"] = scaler.fit_transform(df_attempts[["Total_Scoring_Attempts"]]) * 100
df_assists["Assist_Rating"] = scaler.fit_transform(df_assists[["Assists"]]) * 100


df_goals_r = df_goals[["Player", "Goals_Rating"]]
df_xg_r = df_xg[["Player", "xG_Rating"]]
df_g90_r = df_g90[["Player", "G90_Rating"]]
df_bigch_r = df_bigch[["Player", "BigChance_Rating"]]
df_attempts_r = df_attempts[["Player", "Attempt_Rating"]]
df_assists_r = df_assists[["Player", "Assist_Rating"]]


df_fw = df_goals_r.merge(df_xg_r, on="Player", how="outer") \
                  .merge(df_g90_r, on="Player", how="outer") \
                  .merge(df_bigch_r, on="Player", how="outer") \
                  .merge(df_attempts_r, on="Player", how="outer") \
                  .merge(df_assists_r, on="Player", how="outer")


df_fw.fillna(0, inplace=True)


df_fw["FW_Rating"] = (
    0.25 * df_fw["Goals_Rating"] +
    0.15 * df_fw["xG_Rating"] +
    0.15 * df_fw["G90_Rating"] +
    0.15 * df_fw["BigChance_Rating"] +
    0.15 * df_fw["Attempt_Rating"] +
    0.15 * df_fw["Assist_Rating"]
)




dataframes = [df_goals, df_xg, df_g90, df_bigch, df_attempts, df_assists]


clubs_sources = []
for df in dataframes:
    if "Club" in df.columns:
        clubs_sources.append(df[["Player", "Club"]])


df_clubs = pd.concat(clubs_sources, axis=0)
df_clubs = df_clubs.dropna(subset=["Club"]).drop_duplicates(subset="Player", keep="first")


df_fw = df_fw.merge(df_clubs, on="Player", how="left")


top_forwards = df_fw.sort_values("FW_Rating", ascending=False).head(15).reset_index(drop=True)
top_forwards.insert(0, "Rank", top_forwards.index + 1)
print(top_forwards[["Rank", "Player", "Club", "FW_Rating"]].round(2))


    Rank       Player               Club  FW_Rating
0      1      M.Salah          Liverpool      85.07
1      2    E.Haaland    Manchester City      42.02
2      3     C.Palmer            Chelsea      40.30
3      4       A.Isak   Newcastle United      39.95
4      5      J.Duran      Aston Villa        30.00
5      6     B.Mbeumo          Brentford      28.00
6      7    O.Watkins        Aston Villa      25.76
7      8      Y.Wissa          Brentford      25.50
8      9       B.Saka            Arsenal      23.88
9     10       C.Wood  Nottingham Forest      22.61
10    11      M.Cunha  Manchester United      18.20
11    12      R.Muniz           Fulham        16.59
12    13  B.Fernandes  Manchester United      15.15
13    14     J.Murphy   Newcastle United      14.69
14    15        E.Eze     Crystal Palace      14.54


## Asistler

In [46]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df_assists = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\assists.csv", encoding="ISO-8859-9")
df_accurate = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\accurate_passes_stats.csv", encoding="ISO-8859-9")
df_bigch = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\big_chances_created_stats.csv", encoding="ISO-8859-9")
df_won = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\won_contests_stats.csv", encoding="ISO-8859-9")
df_goals = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\goals.csv", encoding="ISO-8859-9")


scaler = MinMaxScaler()

df_assists["Assist_Rating"] = scaler.fit_transform(df_assists[["Assists"]]) * 100
df_accurate["Accurate_Rating"] = scaler.fit_transform(df_accurate[["Accurate_Passes"]]) * 100
df_bigch["BigChance_Rating"] = scaler.fit_transform(df_bigch[["Big_Chances_Created"]]) * 100
df_won["WonContest_Rating"] = scaler.fit_transform(df_won[["Won_Contests"]]) * 100
df_goals["Goals_Rating"] = scaler.fit_transform(df_goals[["Goals"]]) * 100


df_assists_r = df_assists[["Player", "Assist_Rating"]]
df_accurate_r = df_accurate[["Player", "Accurate_Rating"]]
df_bigch_r = df_bigch[["Player", "BigChance_Rating"]]
df_won_r = df_won[["Player", "WonContest_Rating"]]
df_goals_r = df_goals[["Player", "Goals_Rating"]]


df_mid = df_assists_r.merge(df_accurate_r, on="Player", how="outer") \
                     .merge(df_bigch_r, on="Player", how="outer") \
                     .merge(df_won_r, on="Player", how="outer") \
                     .merge(df_goals_r, on="Player", how="outer")


df_mid.fillna(0, inplace=True)


df_mid["MID_Rating"] = (
    0.3 * df_mid["Assist_Rating"] +
    0.20 * df_mid["Accurate_Rating"] +
    0.35 * df_mid["BigChance_Rating"] +
    0.10 * df_mid["WonContest_Rating"] +
    0.05 * df_mid["Goals_Rating"]
)


top_mid = df_mid.sort_values("MID_Rating", ascending=False).head(15).reset_index(drop=True)
top_mid.insert(0, "Rank", top_mid.index + 1)

print(top_mid[["Rank", "Player", "MID_Rating"]].round(2))


    Rank       Player  MID_Rating
0      1      M.Salah       70.00
1      2     C.Palmer       43.27
2      3       B.Saka       39.39
3      4     J.Murphy       30.94
4      5     M.Rogers       28.03
5      6  B.Fernandes       27.60
6      7     B.Mbeumo       27.54
7      8        H.Son       25.94
8      9  M.Damsgaard       25.42
9     10       I.Sarr       23.12
10    11  Y.Tielemans       22.60
11    12  K.De Bruyne       20.42
12    13       R.Dias       20.00
13    14     A.Elanga       18.33
14    15    T.Cairney       17.79


## Defans

In [40]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


df_interceptions = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\interceptions_stats.csv", encoding="ISO-8859-9")
df_won_tackles = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\won_tackles_stats_fixed.csv", encoding="ISO-8859-9")
df_accurate = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\accurate_passes_stats.csv", encoding="ISO-8859-9")


scaler = MinMaxScaler()

df_interceptions["Interceptions_Rating"] = scaler.fit_transform(df_interceptions[["Interceptions"]]) * 100
df_won_tackles["WonTackles_Rating"] = scaler.fit_transform(df_won_tackles[["Won_Tackles"]]) * 100
df_accurate["Accurate_Rating"] = scaler.fit_transform(df_accurate[["Accurate_Passes"]]) * 100


df_interceptions_r = df_interceptions[["Player", "Interceptions_Rating"]]
df_won_tackles_r = df_won_tackles[["Player", "WonTackles_Rating"]]
df_accurate_r = df_accurate[["Player", "Accurate_Rating"]]


df_def = df_interceptions_r.merge(df_won_tackles_r, on="Player", how="outer") \
                          .merge(df_accurate_r, on="Player", how="outer")


df_def.fillna(0, inplace=True)


df_def["DEF_Rating"] = (
    0.5 * df_def["Interceptions_Rating"] +
    0.45 * df_def["WonTackles_Rating"] +
    0.05 * df_def["Accurate_Rating"]
)


top_def = df_def.sort_values("DEF_Rating", ascending=False).head(15).reset_index(drop=True)
top_def.insert(0, "Rank", top_def.index + 1)

print(top_def[["Rank", "Player", "DEF_Rating"]].round(2))


    Rank         Player  DEF_Rating
0      1         W.Endo       83.17
1      2      M.Wieffer       51.25
2      3    R.Bentancur       50.00
3      4       Casemiro       45.00
4      5     J.Bednarek       37.96
5      6      H.Maguire       37.50
6      7  A.Wan-Bissaka       31.25
7      8      D.Huijsen       31.25
8      9       M.Ugarte       27.69
9     10     A.Robinson       25.00
10    11   K.Mavropanos       25.00
11    12     L.Bergvall       25.00
12    13        T.Adams       20.77
13    14      I.G.Gueye       20.77
14    15        J.Lerma       18.75


## Kaleci

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


df_save = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\save_percentage_stats.csv", encoding="ISO-8859-9")
df_clean = pd.read_csv("C:\\Users\\MONSTER\\Desktop\\fotmob stats\\clean_sheets_stats.csv", encoding="ISO-8859-9")


df_gk = df_save[["Player", "Save_Percentage"]].merge(
    df_clean[["Player", "Clean_Sheets"]], on="Player", how="outer"
)
df_gk.fillna(0, inplace=True)

scaler_gk = MinMaxScaler()
gk_scaled = scaler_gk.fit_transform(df_gk[["Save_Percentage", "Clean_Sheets"]])
df_gk["Save_Rating"] = gk_scaled[:, 0] * 100
df_gk["CleanSheet_Rating"] = gk_scaled[:, 1] * 100


df_gk["GK_Rating"] = 0.5 * df_gk["Save_Rating"] + 0.5 * df_gk["CleanSheet_Rating"]


top_gk = df_gk.sort_values("GK_Rating", ascending=False).head(15).reset_index(drop=True)
top_gk.insert(0, "Rank", top_gk.index + 1)
print(top_gk[["Rank", "Player", "GK_Rating"]].round(2))

    Rank        Player  GK_Rating
0      1        M.Sels      99.12
1      2        D.Raya      98.71
2      3    J.Pickford      96.09
3      4     R.Sanchez      88.46
4      5   D.Henderson      87.89
5      6      A.Becker      84.14
6      7       A.Onana      80.27
7      8        N.Pope      79.34
8      9    E.Martinez      77.64
9     10     M.Flekken      76.92
10    11  B.Verbruggen      68.43
11    12          J.Sa      67.48
12    13        B.Leno      64.82
13    14      A.Areola      60.70
14    15     G.Vicario      59.41


## MVP

In [74]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from functools import reduce

base_path = "C:\\Users\\MONSTER\\Desktop\\fotmob stats\\"


df_save = pd.read_csv(base_path + "save_percentage_stats.csv", encoding="ISO-8859-9")
df_clean = pd.read_csv(base_path + "clean_sheets_stats.csv", encoding="ISO-8859-9")
df_interceptions = pd.read_csv(base_path + "interceptions_stats.csv", encoding="ISO-8859-9")
df_won_tackles = pd.read_csv(base_path + "won_tackles_stats_fixed.csv", encoding="ISO-8859-9")
df_accurate = pd.read_csv(base_path + "accurate_passes_stats.csv", encoding="ISO-8859-9")
df_assists = pd.read_csv(base_path + "assists.csv", encoding="ISO-8859-9")
df_bigch = pd.read_csv(base_path + "big_chances_created_stats.csv", encoding="ISO-8859-9")
df_won = pd.read_csv(base_path + "won_contests_stats.csv", encoding="ISO-8859-9")
df_goals = pd.read_csv(base_path + "goals.csv", encoding="ISO-8859-9")
df_xg = pd.read_csv(base_path + "expected_goals_stats.csv", encoding="ISO-8859-9")
df_g90 = pd.read_csv(base_path + "goals_per_90_stats.csv", encoding="ISO-8859-9")
df_attempts = pd.read_csv(base_path + "total_scoring_attempts_stats.csv", encoding="ISO-8859-9")

scaler = MinMaxScaler()


df_fw = df_goals[["Player", "Goals"]].merge(
    df_g90[["Player", "Goals_Per_90"]], on="Player", how="outer"
).merge(
    df_assists[["Player", "Assists"]], on="Player", how="outer"
).merge(
    df_bigch[["Player", "Big_Chances_Created"]], on="Player", how="outer"
).merge(
    df_attempts[["Player", "Total_Scoring_Attempts"]], on="Player", how="outer"
)
df_fw.fillna(0, inplace=True)

for col in ["Goals", "Goals_Per_90", "Assists", "Big_Chances_Created", "Total_Scoring_Attempts"]:
    df_fw[col + "_norm"] = scaler.fit_transform(df_fw[[col]]) * 100

df_fw["FW_Rating"] = (
    0.3 * df_fw["Goals_norm"] +
    0.25 * df_fw["Goals_Per_90_norm"] +
    0.2 * df_fw["Assists_norm"] +
    0.15 * df_fw["Big_Chances_Created_norm"] +
    0.1 * df_fw["Total_Scoring_Attempts_norm"]
)
fw_mvp = df_fw.sort_values("FW_Rating", ascending=False).head(1)[["Player", "FW_Rating"]]


df_mid = df_assists[["Player", "Assists"]].merge(
    df_accurate[["Player", "Accurate_Passes"]], on="Player", how="outer"
).merge(
    df_bigch[["Player", "Big_Chances_Created"]], on="Player", how="outer"
).merge(
    df_won[["Player", "Won_Contests"]], on="Player", how="outer"
).merge(
    df_goals[["Player", "Goals"]], on="Player", how="outer"
)
df_mid.fillna(0, inplace=True)

for col in ["Assists", "Accurate_Passes", "Big_Chances_Created", "Won_Contests", "Goals"]:
    df_mid[col + "_norm"] = scaler.fit_transform(df_mid[[col]]) * 100

df_mid["MID_Rating"] = (
    0.3 * df_mid["Assists_norm"] +
    0.25 * df_mid["Accurate_Passes_norm"] +
    0.2 * df_mid["Big_Chances_Created_norm"] +
    0.15 * df_mid["Won_Contests_norm"] +
    0.1 * df_mid["Goals_norm"]
)
mid_mvp = df_mid.sort_values("MID_Rating", ascending=False).head(1)[["Player", "MID_Rating"]]


df_def = df_interceptions[["Player", "Interceptions"]].merge(
    df_won_tackles[["Player", "Won_Tackles"]], on="Player", how="outer"
).merge(
    df_accurate[["Player", "Accurate_Passes"]], on="Player", how="outer"
)
df_def.fillna(0, inplace=True)

for col in ["Interceptions", "Won_Tackles", "Accurate_Passes"]:
    df_def[col + "_norm"] = scaler.fit_transform(df_def[[col]]) * 100

df_def["DEF_Rating"] = (
    0.4 * df_def["Interceptions_norm"] +
    0.35 * df_def["Won_Tackles_norm"] +
    0.25 * df_def["Accurate_Passes_norm"]
)
def_mvp = df_def.sort_values("DEF_Rating", ascending=False).head(1)[["Player", "DEF_Rating"]]


df_gk = df_save[["Player", "Save_Percentage"]].merge(
    df_clean[["Player", "Clean_Sheets"]], on="Player", how="outer"
)
df_gk.fillna(0, inplace=True)

scaler_gk = MinMaxScaler()
gk_scaled = scaler_gk.fit_transform(df_gk[["Save_Percentage", "Clean_Sheets"]])
df_gk["Save_Rating"] = gk_scaled[:, 0] * 100
df_gk["CleanSheet_Rating"] = gk_scaled[:, 1] * 100

df_gk["GK_Rating"] = 0.5 * df_gk["Save_Rating"] + 0.5 * df_gk["CleanSheet_Rating"]
gk_mvp = df_gk.sort_values("GK_Rating", ascending=False).head(1)[["Player", "GK_Rating"]]


mvp_dfs = [
    fw_mvp.rename(columns={"FW_Rating": "Rating"}),
    mid_mvp.rename(columns={"MID_Rating": "Rating"}),
    def_mvp.rename(columns={"DEF_Rating": "Rating"}),
    gk_mvp.rename(columns={"GK_Rating": "Rating"})
]
mvp_players = pd.concat(mvp_dfs, keys=["FW", "MID", "DEF", "GK"]).reset_index(level=0).rename(columns={"level_0": "Position"})

print("Mevki Bazlı MVP'ler:")
print(mvp_players)


dfs = [
    df_goals, df_assists, df_interceptions, df_won_tackles, df_accurate,
    df_bigch, df_won, df_save, df_clean, df_xg, df_g90, df_attempts
]
stat_cols = [
    "Goals", "Assists", "Interceptions", "Won_Tackles", "Accurate_Passes",
    "Big_Chances_Created", "Won_Contests", "Save_Percentage", "Clean_Sheets",
    "Expected_Goals", "Goals_Per_90", "Total_Scoring_Attempts"
]

dfs_trimmed = [df[["Player", col]] for df, col in zip(dfs, stat_cols)]
df_all_stats = reduce(lambda left, right: pd.merge(left, right, on="Player", how="outer"), dfs_trimmed)
df_all_stats.fillna(0, inplace=True)

df_mvp_stats = df_all_stats[df_all_stats["Player"].isin(mvp_players["Player"])].copy()

for col in stat_cols:
    df_mvp_stats[col] = scaler.fit_transform(df_mvp_stats[[col]]) * 100

weight = 1 / len(stat_cols)
df_mvp_stats["General_MVP_Score"] = sum(df_mvp_stats[col] * weight for col in stat_cols)

general_mvp = df_mvp_stats.sort_values("General_MVP_Score", ascending=False).head(1)

print("\nGenel MVP:")
print(general_mvp[["Player", "General_MVP_Score"]].round(2))


Mevki Bazlı MVP'ler:
  Position   Player     Rating
0       FW  M.Salah  91.837184
0      MID  M.Salah  60.000000
1      DEF   W.Endo  88.718101
4       GK   M.Sels  99.116848

Genel MVP:
    Player  General_MVP_Score
0  M.Salah               50.0
